In [ ]:
import random
import pickle
from collections import Counter

# -------------------------
# 1️⃣ Load and clean corpus
# -------------------------
with open("corpus.txt", "r", encoding="utf-8") as f:
    text = f.read()

# Normalize text
text = text.replace('', ' ')
text = text.replace('\n', ' ')
text = ' '.join(text.split())

# Split into sentences (roughly on Urdu full stop / question mark)
sentences = [s.strip() for s in text.split('۔') if s.strip()]
processed_sentences = []

# Add <BOS> and <EOT> to each sentence
for s in sentences:
    tokens = s.split()
    if tokens:
        processed_sentences.append(['<BOS>', '<BOS>'] + tokens + ['<EOT>'])

# Flatten all sentences for trigram counting
all_tokens = [token for sentence in processed_sentences for token in sentence]

# -------------------------
# 2️⃣ Build n-gram counts
# -------------------------
unigrams = Counter(all_tokens)
bigrams = Counter(zip(all_tokens[:-1], all_tokens[1:]))
trigrams = Counter(zip(all_tokens[:-2], all_tokens[1:-1], all_tokens[2:]))

vocab_size = len(unigrams)
total_tokens = sum(unigrams.values())

# -------------------------
# 3️⃣ Interpolated trigram probability
# -------------------------
lambda1 = 0.2
lambda2 = 0.3
lambda3 = 0.5

def interpolated_prob(w1, w2, w3):
    # MLE probabilities
    tri_count = trigrams.get((w1, w2, w3), 0)
    bi_count = bigrams.get((w1, w2), 0)
    bi_count2 = bigrams.get((w2, w3), 0)
    uni_count = unigrams.get(w3, 0)
    
    p_tri = tri_count / bi_count if bi_count > 0 else 0
    p_bi = bi_count2 / unigrams[w2] if unigrams[w2] > 0 else 0
    p_uni = uni_count / total_tokens
    
    return lambda3*p_tri + lambda2*p_bi + lambda1*p_uni

# -------------------------
# 4️⃣ Generate one sentence
# -------------------------
def generate_sentence(max_words=50):
    sentence = ['<BOS>', '<BOS>']
    
    for _ in range(max_words):
        w1, w2 = sentence[-2], sentence[-1]
        
        candidates = list(unigrams.keys())
        probs = [interpolated_prob(w1, w2, w3) for w3 in candidates]
        total = sum(probs)
        if total == 0:
            break
        probs = [p/total for p in probs]
        
        next_word = random.choices(candidates, probs)[0]
        sentence.append(next_word)
        
        if next_word == '<EOT>':
            break
    
    # Remove special tokens
    sentence = [w for w in sentence if w not in ['<BOS>', '<EOT>']]
    return ' '.join(sentence)

# -------------------------
# 5️⃣ Generate full paragraph/story
# -------------------------
def generate_story(max_sentences=7):
    story = []
    for _ in range(max_sentences):
        sent = generate_sentence()
        if sent.strip():
            story.append(sent + '۔')  # Urdu full stop
    return ' '.join(story)

# -------------------------
# 6️⃣ Generate multiple stories
# -------------------------
for i in range(3):
    print(f"--- Generated Story {i+1} ---")
    print(generate_story(max_sentences=7))
    print('\n')
    
model_data = {
    "unigrams": unigrams,
    "bigrams": bigrams,
    "trigrams": trigrams,
    "total_tokens": total_tokens
}

with open("model.pkl", "wb") as f:
    pickle.dump(model_data, f)

print("model.pkl saved successfully!")

--- Generated Story 1 ---
تو بابا مجھے اپنا حصہ دار سے ان کے حواس پر قابو کرتی کی ہم بڑی خرابی کی شادیاں سوہنے گھبروجوان تنہائی۔ کرتے ہیں جو اپنی تین جا رہا تھا مجھے پریشانی ہورہی ہیں پہ نہیں بولا۔ بس ذرا لڑائی جھگڑے فساد سے نکل حیرت سے اپنے گھر کے مارے ہلکی سی ٹوٹی ہوئی میں نے بتایا تھا گھر میں استانی۔ ہمارے لئے قائم کو بھسم کرنے کا موقع نہ ہی کمرے میں پہنچیں تو مرسڈیز ہے وہ سمجھیں شروع ہوتا ہے وہ میری طرف پلٹی۔ میں بگھی کا شعلہ نکلتا ہے اور امی کا گیٹ کھول دیا۔ آپ نے انھیں ان ہوتا نہیں ایسے ہی میں جب کھانا کھانے کو کچھ سکے دئیے۔ بچو! علامہ اقبال نے سیٹھ کے کمرے کی تلاش کوئی جا رہے۔


--- Generated Story 2 ---
وہ واپس آتا میں ڈالی لئے مٹھو جیسا تو نہیں کیا کرنا نے ماں کو بھی یہ خوشبو بے وقوف نے ناریل کے گرنے آپ بڑی سال نہ رواں دواں ہو اور طالب علم جو کام خطرناک ہو وہ ہوا اور وہ بے ہوش پر خریدنے آیا۔ اتنے میں دوسرے بچے ، اسی لیے تھا۔ وہیں برسوں کی محنت چلاتے اور کسی مکھی نے جب چمگادڑ کا معمول میں تبدیلی کروگے؟ دیر گھی کی خوشبو اسے بے حد دُکھ کے ٹریننگ سینٹر غرض ہوتی ہیں، غریب کل رہا چل